In [1]:
from sklearn.preprocessing import OrdinalEncoder, PolynomialFeatures, LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from pycaret.regression import setup, compare_models
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.linear_model import BayesianRidge, OrthogonalMatchingPursuit, LinearRegression, Lasso, Ridge
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [3]:
from category_encoders.cat_boost import CatBoostEncoder

# Read train and test sets

In [3]:
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

# Store both dfs lengths

In [4]:
m_train = df_train.shape[0]
m_test = df_test.shape[0]
m_train, m_test

(1460, 1459)

# Concat the dfs

In [5]:
df = pd.concat([df_train, df_test])
assert df.shape[0] == m_train + m_test

In [6]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0


# Save target

In [7]:
target_col, target = "SalePrice", df["SalePrice"]

#  Drop Id and target columns

In [8]:
cols_to_drop = ["Id", target_col]
df.drop(cols_to_drop, axis=1, inplace=True)

# View and fix columns data-types

In [9]:
df.select_dtypes(object).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [10]:
df.select_dtypes(np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold'],
      dtype='object')

In [11]:
num_to_obj_cols = ['MSSubClass', 'MoSold']
df[num_to_obj_cols] = df[num_to_obj_cols].astype(object)

In [12]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute categorical columns

In [13]:
cols_cat_na = df[cols_cat].isnull().sum()[df[cols_cat].isnull().sum() > 0]
cols_cat_na

MSZoning           4
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinType2      80
Electrical         1
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageFinish     159
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

In [14]:
mode_filled_cols = ["MSZoning", "Utilities", "Exterior1st", "Exterior2nd", "MasVnrType", "Electrical", "KitchenQual", "Functional", "SaleType"]
for col in mode_filled_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

none_filled_cols = ["Alley", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "FireplaceQu", "GarageType", "GarageFinish", "GarageQual", "GarageCond", "PoolQC", "Fence", "MiscFeature"]
for col in none_filled_cols:
    df[col].fillna("None", inplace=True)
    
df[cols_cat].isnull().sum().sum()

0

# Change ordinal columns to numeric, and encode accordingly

In [15]:
df.select_dtypes(object).columns

Index(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
       'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond',
       'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
       'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType',
       'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC',
       'Fence', 'MiscFeature', 'MoSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [16]:
oe = OrdinalEncoder(categories=[['Reg', 'IR1', 'IR2', 'IR3']])
df.loc[:, "LotShape"] = oe.fit_transform(df[["LotShape"]])
# df["LotShape_sq"] = df["LotShape"] ** 2

In [17]:
oe = OrdinalEncoder(categories=[['Gtl', 'Mod', 'Sev']])
df.loc[:, "LandSlope"] = oe.fit_transform(df[["LandSlope"]])
# df["LandSlope_sq"] = df["LandSlope"] ** 2

In [18]:
qual_oe = OrdinalEncoder(categories=[['None', 'Po', 'Fa', 'TA', 'Gd', 'Ex']])
for col in ["ExterQual", "ExterCond", 'BsmtQual', 'BsmtCond', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC']:
    df.loc[:, col] = qual_oe.fit_transform(df[[col]])
#     df[col + "sq"] = df[col] ** 2

In [19]:
oe = OrdinalEncoder(categories=[['None', 'No', 'Mn', 'Av', 'Gd']])
df.loc[:, "BsmtExposure"] = oe.fit_transform(df[["BsmtExposure"]])
# df["BsmtExposure_sq"] = df["BsmtExposure"] ** 2

In [20]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']])
for col in ["BsmtFinType1", "BsmtFinType2"]:
    df.loc[:, col] = oe.fit_transform(df[[col]])
#     df[col + "sq"] = df[col] ** 2

In [21]:
oe = OrdinalEncoder(categories=[['None', 'Unf', 'RFn', 'Fin']])
df.loc[:, "GarageFinish"] = oe.fit_transform(df[["GarageFinish"]])
# df["GarageFinishsq"] = df["GarageFinish"] ** 2

In [22]:
oe = OrdinalEncoder(categories=[['N', 'P', 'Y']])
df.loc[:, "PavedDrive"] = oe.fit_transform(df[["PavedDrive"]])
# df["PavedDrivesq"] = df["PavedDrive"] ** 2

In [23]:
oe = OrdinalEncoder(categories=[['None', 'MnWw', 'GdWo', 'MnPrv', 'GdPrv']])
df.loc[:, "Fence"] = oe.fit_transform(df[["Fence"]])
# df["Fencesq"] = df["Fence"] ** 2

In [24]:
cols_cat = df.select_dtypes(object).columns.to_list()
cols_num = df.select_dtypes(np.number).columns.to_list()

# Impute numerical columns

In [25]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage     486
MasVnrArea       23
BsmtFinSF1        1
BsmtFinSF2        1
BsmtUnfSF         1
TotalBsmtSF       1
BsmtFullBath      2
BsmtHalfBath      2
GarageYrBlt     159
GarageCars        1
GarageArea        1
dtype: int64

In [26]:
zero_filled_cols = ["BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF", "TotalBsmtSF", "BsmtFullBath", "BsmtHalfBath", "GarageCars", "GarageArea"]
for col in zero_filled_cols:
    df[col].fillna(0, inplace=True)

In [27]:
cols_num_na = df[cols_num].isnull().sum()[df[cols_num].isnull().sum() > 0]
cols_num_na

LotFrontage    486
MasVnrArea      23
GarageYrBlt    159
dtype: int64

In [28]:
def catboost_imputer(df, cols_num_na):
    
    # save columns to impute and drop them from df
    cols_to_impute = df[cols_num_na]
    df.drop(cols_num_na, axis=1, inplace=True)
    
    for col in cols_num_na:
        # define X and y
        X, y = df.copy(), cols_to_impute[col]

        # get train and test sets
        train_indexes, test_indexes = ~y.isnull(), y.isnull()
        X_train, X_test = X.loc[train_indexes, :], X.loc[test_indexes, :]
        y_train, y_test = y.loc[train_indexes], y.loc[test_indexes]
        
        model = CatBoostRegressor(max_depth=8, random_seed=10,
                                  subsample=0.65, n_estimators=1000,
                                  cat_features=cols_cat)
        model.fit(X_train, y_train)
        cols_to_impute.loc[test_indexes, col] = model.predict(X_test)
        df[col] = cols_to_impute[col]
    return df

In [29]:
df = catboost_imputer(df, cols_num_na.index)
df.isnull().sum().sum()

Learning rate set to 0.047118
0:	learn: 22.8881607	total: 60ms	remaining: 59.9s
1:	learn: 22.4167077	total: 83.5ms	remaining: 41.7s
2:	learn: 21.9674201	total: 114ms	remaining: 37.7s
3:	learn: 21.5550892	total: 126ms	remaining: 31.4s
4:	learn: 21.1728677	total: 142ms	remaining: 28.3s
5:	learn: 20.8205778	total: 156ms	remaining: 25.8s
6:	learn: 20.4210801	total: 174ms	remaining: 24.6s
7:	learn: 20.0631293	total: 190ms	remaining: 23.6s
8:	learn: 19.7546986	total: 207ms	remaining: 22.8s
9:	learn: 19.4290106	total: 227ms	remaining: 22.4s
10:	learn: 19.1155800	total: 240ms	remaining: 21.6s
11:	learn: 18.8276333	total: 260ms	remaining: 21.4s
12:	learn: 18.5296271	total: 276ms	remaining: 21s
13:	learn: 18.2768483	total: 293ms	remaining: 20.6s
14:	learn: 18.0281784	total: 307ms	remaining: 20.1s
15:	learn: 17.7467394	total: 325ms	remaining: 20s
16:	learn: 17.4911819	total: 340ms	remaining: 19.7s
17:	learn: 17.2760921	total: 356ms	remaining: 19.4s
18:	learn: 17.0819915	total: 373ms	remaining: 19

168:	learn: 10.8996323	total: 2.68s	remaining: 13.2s
169:	learn: 10.8759552	total: 2.7s	remaining: 13.2s
170:	learn: 10.8610464	total: 2.71s	remaining: 13.2s
171:	learn: 10.8501047	total: 2.73s	remaining: 13.2s
172:	learn: 10.8206732	total: 2.75s	remaining: 13.1s
173:	learn: 10.7984382	total: 2.76s	remaining: 13.1s
174:	learn: 10.7626695	total: 2.77s	remaining: 13.1s
175:	learn: 10.7626230	total: 2.78s	remaining: 13s
176:	learn: 10.7562282	total: 2.79s	remaining: 13s
177:	learn: 10.7410752	total: 2.8s	remaining: 12.9s
178:	learn: 10.6987336	total: 2.82s	remaining: 12.9s
179:	learn: 10.6868306	total: 2.83s	remaining: 12.9s
180:	learn: 10.6578630	total: 2.85s	remaining: 12.9s
181:	learn: 10.6432720	total: 2.86s	remaining: 12.9s
182:	learn: 10.6254567	total: 2.88s	remaining: 12.9s
183:	learn: 10.6181505	total: 2.9s	remaining: 12.9s
184:	learn: 10.6100836	total: 2.92s	remaining: 12.9s
185:	learn: 10.6044391	total: 2.93s	remaining: 12.8s
186:	learn: 10.6040953	total: 2.95s	remaining: 12.8s


327:	learn: 8.8421768	total: 5.22s	remaining: 10.7s
328:	learn: 8.8271954	total: 5.24s	remaining: 10.7s
329:	learn: 8.8079449	total: 5.25s	remaining: 10.7s
330:	learn: 8.8073084	total: 5.27s	remaining: 10.7s
331:	learn: 8.7998563	total: 5.29s	remaining: 10.6s
332:	learn: 8.7874218	total: 5.31s	remaining: 10.6s
333:	learn: 8.7758663	total: 5.32s	remaining: 10.6s
334:	learn: 8.7667110	total: 5.34s	remaining: 10.6s
335:	learn: 8.7518400	total: 5.35s	remaining: 10.6s
336:	learn: 8.7429349	total: 5.37s	remaining: 10.6s
337:	learn: 8.7320831	total: 5.38s	remaining: 10.5s
338:	learn: 8.7310364	total: 5.4s	remaining: 10.5s
339:	learn: 8.7250007	total: 5.41s	remaining: 10.5s
340:	learn: 8.7193640	total: 5.43s	remaining: 10.5s
341:	learn: 8.7188219	total: 5.45s	remaining: 10.5s
342:	learn: 8.7101290	total: 5.46s	remaining: 10.5s
343:	learn: 8.6968252	total: 5.48s	remaining: 10.5s
344:	learn: 8.6754105	total: 5.5s	remaining: 10.4s
345:	learn: 8.6633886	total: 5.51s	remaining: 10.4s
346:	learn: 8.

489:	learn: 7.5323886	total: 7.95s	remaining: 8.27s
490:	learn: 7.5234405	total: 7.96s	remaining: 8.25s
491:	learn: 7.5231053	total: 7.98s	remaining: 8.24s
492:	learn: 7.5166388	total: 7.99s	remaining: 8.22s
493:	learn: 7.5145724	total: 8.01s	remaining: 8.2s
494:	learn: 7.5116597	total: 8.02s	remaining: 8.19s
495:	learn: 7.5057394	total: 8.04s	remaining: 8.17s
496:	learn: 7.5035762	total: 8.06s	remaining: 8.15s
497:	learn: 7.5009852	total: 8.08s	remaining: 8.14s
498:	learn: 7.4921159	total: 8.09s	remaining: 8.12s
499:	learn: 7.4892158	total: 8.1s	remaining: 8.1s
500:	learn: 7.4855485	total: 8.12s	remaining: 8.09s
501:	learn: 7.4701740	total: 8.13s	remaining: 8.07s
502:	learn: 7.4622350	total: 8.15s	remaining: 8.05s
503:	learn: 7.4530071	total: 8.16s	remaining: 8.04s
504:	learn: 7.4490366	total: 8.18s	remaining: 8.02s
505:	learn: 7.4445984	total: 8.2s	remaining: 8.01s
506:	learn: 7.4439574	total: 8.21s	remaining: 7.99s
507:	learn: 7.4420778	total: 8.23s	remaining: 7.97s
508:	learn: 7.44

657:	learn: 6.5424336	total: 10.7s	remaining: 5.55s
658:	learn: 6.5305424	total: 10.7s	remaining: 5.53s
659:	learn: 6.5219083	total: 10.7s	remaining: 5.52s
660:	learn: 6.5205178	total: 10.7s	remaining: 5.5s
661:	learn: 6.5112673	total: 10.7s	remaining: 5.49s
662:	learn: 6.5102018	total: 10.8s	remaining: 5.47s
663:	learn: 6.5099631	total: 10.8s	remaining: 5.45s
664:	learn: 6.5096090	total: 10.8s	remaining: 5.43s
665:	learn: 6.4998151	total: 10.8s	remaining: 5.42s
666:	learn: 6.4974830	total: 10.8s	remaining: 5.4s
667:	learn: 6.4874702	total: 10.8s	remaining: 5.38s
668:	learn: 6.4839869	total: 10.8s	remaining: 5.37s
669:	learn: 6.4836687	total: 10.9s	remaining: 5.35s
670:	learn: 6.4812258	total: 10.9s	remaining: 5.33s
671:	learn: 6.4800726	total: 10.9s	remaining: 5.32s
672:	learn: 6.4797120	total: 10.9s	remaining: 5.3s
673:	learn: 6.4771672	total: 10.9s	remaining: 5.29s
674:	learn: 6.4686327	total: 10.9s	remaining: 5.27s
675:	learn: 6.4582340	total: 11s	remaining: 5.25s
676:	learn: 6.457

...

467:	learn: 5.6943350	total: 7.94s	remaining: 9.03s
468:	learn: 5.6804518	total: 7.96s	remaining: 9.01s
469:	learn: 5.6769710	total: 7.98s	remaining: 8.99s
470:	learn: 5.6728753	total: 7.99s	remaining: 8.97s
471:	learn: 5.6670686	total: 8.01s	remaining: 8.96s
472:	learn: 5.6601472	total: 8.02s	remaining: 8.94s
473:	learn: 5.6589766	total: 8.05s	remaining: 8.93s
474:	learn: 5.6585080	total: 8.07s	remaining: 8.92s
475:	learn: 5.6536945	total: 8.11s	remaining: 8.93s
476:	learn: 5.6523018	total: 8.14s	remaining: 8.93s
477:	learn: 5.6517953	total: 8.17s	remaining: 8.92s
478:	learn: 5.6458331	total: 8.19s	remaining: 8.9s
479:	learn: 5.6382947	total: 8.21s	remaining: 8.89s
480:	learn: 5.6245885	total: 8.23s	remaining: 8.88s
481:	learn: 5.6203263	total: 8.26s	remaining: 8.88s
482:	learn: 5.6069048	total: 8.28s	remaining: 8.86s
483:	learn: 5.6038583	total: 8.31s	remaining: 8.86s
484:	learn: 5.6000069	total: 8.34s	remaining: 8.85s
485:	learn: 5.5921729	total: 8.36s	remaining: 8.84s
486:	learn: 5

633:	learn: 4.8487142	total: 11.1s	remaining: 6.4s
634:	learn: 4.8461358	total: 11.1s	remaining: 6.38s
635:	learn: 4.8437895	total: 11.1s	remaining: 6.37s
636:	learn: 4.8329397	total: 11.1s	remaining: 6.35s
637:	learn: 4.8328096	total: 11.2s	remaining: 6.33s
638:	learn: 4.8293535	total: 11.2s	remaining: 6.32s
639:	learn: 4.8167721	total: 11.2s	remaining: 6.3s
640:	learn: 4.8069292	total: 11.2s	remaining: 6.28s
641:	learn: 4.8051040	total: 11.2s	remaining: 6.27s
642:	learn: 4.8003684	total: 11.3s	remaining: 6.25s
643:	learn: 4.7997923	total: 11.3s	remaining: 6.24s
644:	learn: 4.7977912	total: 11.3s	remaining: 6.22s
645:	learn: 4.7929458	total: 11.3s	remaining: 6.21s
646:	learn: 4.7866459	total: 11.3s	remaining: 6.19s
647:	learn: 4.7856378	total: 11.4s	remaining: 6.17s
648:	learn: 4.7831790	total: 11.4s	remaining: 6.15s
649:	learn: 4.7735367	total: 11.4s	remaining: 6.14s
650:	learn: 4.7705674	total: 11.4s	remaining: 6.12s
651:	learn: 4.7693257	total: 11.4s	remaining: 6.1s
652:	learn: 4.7

802:	learn: 4.1158614	total: 14.2s	remaining: 3.48s
803:	learn: 4.1085214	total: 14.2s	remaining: 3.46s
804:	learn: 4.1072425	total: 14.2s	remaining: 3.44s
805:	learn: 4.0996797	total: 14.2s	remaining: 3.43s
806:	learn: 4.0947219	total: 14.3s	remaining: 3.41s
807:	learn: 4.0858265	total: 14.3s	remaining: 3.39s
808:	learn: 4.0853119	total: 14.3s	remaining: 3.37s
809:	learn: 4.0797151	total: 14.3s	remaining: 3.36s
810:	learn: 4.0685375	total: 14.3s	remaining: 3.34s
811:	learn: 4.0625737	total: 14.3s	remaining: 3.32s
812:	learn: 4.0510531	total: 14.4s	remaining: 3.3s
813:	learn: 4.0499731	total: 14.4s	remaining: 3.28s
814:	learn: 4.0403422	total: 14.4s	remaining: 3.27s
815:	learn: 4.0400853	total: 14.4s	remaining: 3.25s
816:	learn: 4.0334452	total: 14.4s	remaining: 3.23s
817:	learn: 4.0301965	total: 14.4s	remaining: 3.21s
818:	learn: 4.0254428	total: 14.5s	remaining: 3.2s
819:	learn: 4.0251679	total: 14.5s	remaining: 3.18s
820:	learn: 4.0249632	total: 14.5s	remaining: 3.16s
821:	learn: 4.

969:	learn: 3.4810365	total: 17.1s	remaining: 529ms
970:	learn: 3.4766565	total: 17.1s	remaining: 512ms
971:	learn: 3.4698376	total: 17.1s	remaining: 494ms
972:	learn: 3.4694080	total: 17.2s	remaining: 476ms
973:	learn: 3.4670027	total: 17.2s	remaining: 459ms
974:	learn: 3.4666406	total: 17.2s	remaining: 441ms
975:	learn: 3.4658913	total: 17.2s	remaining: 423ms
976:	learn: 3.4597789	total: 17.2s	remaining: 406ms
977:	learn: 3.4568824	total: 17.2s	remaining: 388ms
978:	learn: 3.4552582	total: 17.3s	remaining: 370ms
979:	learn: 3.4496646	total: 17.3s	remaining: 353ms
980:	learn: 3.4435802	total: 17.3s	remaining: 335ms
981:	learn: 3.4420173	total: 17.3s	remaining: 317ms
982:	learn: 3.4418044	total: 17.3s	remaining: 300ms
983:	learn: 3.4417702	total: 17.3s	remaining: 282ms
984:	learn: 3.4415052	total: 17.4s	remaining: 264ms
985:	learn: 3.4382695	total: 17.4s	remaining: 247ms
986:	learn: 3.4347745	total: 17.4s	remaining: 229ms
987:	learn: 3.4346734	total: 17.4s	remaining: 212ms
988:	learn: 

0

# Feature selection

In [30]:
def get_del_cols(df):
    # return the columns where no more than one value in the categorical features
    # exists in the test data - those features can be ignored
    cols_to_drop = []
    for col in df.select_dtypes(object).columns:
        col_vals = df[col].unique()
        n_vals = len(col_vals)
        n_irrelevant = 0
        for val in col_vals:
            if val not in df[col][m_train:].values:
                n_irrelevant += 1
        if n_irrelevant >= n_vals - 1:
            cols_to_drop.append(col)
    return cols_to_drop

In [31]:
cols_to_drop = get_del_cols(df)
df.drop(cols_to_drop, axis=1, inplace=True)
for col_to_drop in cols_to_drop:
    cols_cat.remove(col_to_drop)
    print("Dropped ", col_to_drop)

Dropped  Utilities


<!-- # Change ordinal columns to numeric, and encode accordingly -->

# Feature Engineering

## Features from the Internet - prefex FE0_

In [32]:
df["FE0_HighQualSF"] = df["GrLivArea"]+df["1stFlrSF"] + df["2ndFlrSF"]+0.5*df["GarageArea"]+0.5*df["TotalBsmtSF"]+1*df["MasVnrArea"]

df["FE0_SqFtPerRoom"] = df["GrLivArea"] / (df["TotRmsAbvGrd"] +
                                           df["FullBath"] +
                                           df["HalfBath"] +
                                           df["KitchenAbvGr"])

## My features - prefex FE1_

In [33]:
n_stories_dict = {
    "1Story": 1.0,
    "1.5Fin": 1.5,
    "1.5Unf": 1.5,
    "2Story": 2.0,
    "2.5Fin": 2.5,
    "2.5Unf": 2.5,
    "SFoyer": 2.0,
    "SLvl": 2.0,
}

df["FE1_n_stories"] = df["HouseStyle"].replace(n_stories_dict)
df["FE1_n_stories"].value_counts()

1.0    1471
2.0    1083
1.5     333
2.5      32
Name: FE1_n_stories, dtype: int64

In [34]:
df["FE1_age_sold"] = df["YrSold"] - df["YearBuilt"]
df["FE1_age_sold_Remod"] = df["YrSold"] - df["YearRemodAdd"]
df["FE1_GarageYrSold"] = df["YrSold"] - df["GarageYrBlt"]

In [35]:
df["CentralAir"] = df["CentralAir"].replace({"Y": 1, "N": 0})
df["CentralAir"].value_counts()

1    2723
0     196
Name: CentralAir, dtype: int64

In [36]:
df['FE1_total_area_hq'] = df["FE0_HighQualSF"] - df['LowQualFinSF']

In [37]:
df.select_dtypes(np.number).columns

Index(['LotArea', 'LotShape', 'LandSlope', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'ExterQual', 'ExterCond', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'HeatingQC',
       'CentralAir', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea',
       'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Fireplaces',
       'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC',
       'Fence', 'MiscVal', 'YrSold', 'LotFrontage', 'MasVnrArea',
       'GarageYrBlt', 'FE0_HighQualSF', 'FE0_SqFtPerRoom', 'FE1_n_stories',
       'FE1_age_sold', 'FE1_age_sold_Remod', 'FE1_GarageYrSold',
       'FE1_total_area_hq'],
      dtype='object')

## Apply polynomial features

In [38]:
cols_to_poly = ['LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'ExterQual', 'ExterCond','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1','BsmtFinType2', 'BsmtFinSF2', 'TotalBsmtSF', 'FE1_total_area_hq', 'GrLivArea', 'FullBath', 'HalfBath', 'FE0_SqFtPerRoom', 'KitchenQual', 'KitchenAbvGr', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PoolArea', 'YrSold', 'LotFrontage', 'FE0_HighQualSF',]

In [39]:
poly = PolynomialFeatures(degree=2, include_bias=False)
cols_df = df[cols_to_poly]
df.drop(cols_to_poly, axis=1, inplace=True)
cols_array = poly.fit_transform(cols_df)
cols_df = pd.DataFrame(cols_array, columns=poly.get_feature_names_out())
df = pd.concat((df.reset_index(drop=True), cols_df.reset_index(drop=True)), axis=1)

df.shape

(2919, 613)

# Split test and train

In [40]:
X_train, X_test = df[:m_train], df[m_train:]
y_train, _ = target[:m_train], target[m_train:]
n = X_train.shape[1]
X_train.shape, y_train.shape

((1460, 613), (1460,))

In [41]:
df_ohe = pd.get_dummies(df)

X_train_ohe, X_test_ohe = df_ohe[:m_train], df_ohe[m_train:]
X_train_ohe.shape, y_train.shape

((1460, 790), (1460,))

In [51]:
# temp = df_ohe.copy()
# temp[target_col] = target.reset_index(drop=True)
# _ = setup(data=temp[:m_train], target='SalePrice')
# compare_models()

# CatBoost Grid Search 

In [43]:
# cat_features = df.select_dtypes(object).columns.to_list()
# model = CatBoostRegressor(random_state=10, cat_features=cat_features)
# kwargs = {
#     "n_estimators": [50 * n],
#     "max_depth": [8],
#     "subsample": [0.65],
#     "reg_lambda": [0.1], 
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# CatBoost Params:
    {'max_depth': 8, 'n_estimators': 11000 (the more the better), 'reg_lambda': 0.1, 'subsample': 0.65}

# GBR Grid Search 

In [44]:
# model = GradientBoostingRegressor(random_state=10, loss='absolute_error',)
# kwargs = {
#     "learning_rate": [0.1],
#     "n_estimators": [11_000],
#     "max_depth": [3],
#     "subsample": [0.65],
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train_ohe, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# GBR Params:
    {'max_depth': 3, 'n_estimators': the more the better, 'subsample': 0.65}

# XGB Grid Search 

In [45]:
# model = XGBRegressor(eval_metric=mean_squared_error, random_state=10)

# kwargs = {
#     "n_estimators": [2 * n],
#     "max_depth": [3],
#     "learning_rate": [0.1],
#     "subsample": [0.65],
#     "colsample_bytree": [0.5],
#     "reg_lambda": [0.1], 
#     "min_child_weight": [1e-9] 
# }
# clf = GridSearchCV(model, kwargs, verbose=1, n_jobs=2)
# clf.fit(X_train_ohe, y_train)
# print(clf.best_score_)
# print(clf.best_params_)

# XGB Params:
    {'max_depth': 3, 'learning_rate': 0.1, 'n_estimators': the more the better, 'subsample': 0.65, 
     'colsample_bytree': 0.5, 'reg_lambda': 0.1, 'min_child_weight': 1e-9 (the lower the better?)}

# Training all models with Best Parameters

In [46]:
cat_boost_kwards = {
    "max_depth": 8, 
    "learning_rate": 0.1,
    "n_estimators": 11_000,
    "subsample": 0.65, 
    "cat_features": cols_cat,
    "random_state": 10,
}

gbr_kwards = {
    "max_depth": 3,
    "learning_rate": 0.1,
    "n_estimators": 11_000,
    "subsample": 0.65,
    "loss": 'absolute_error',
    "random_state": 10,
}

xgb_kwargs = {
    "max_depth": 3,
    "learning_rate": 0.1,
    "n_estimators": 11_000,
    "subsample": 0.65,
    "colsample_bytree": 0.5,
    "reg_lambda": 0.1, 
    "min_child_weight": 1e-9,
    "eval_metric": mean_squared_error,
    "random_state": 10,
}



In [47]:
cat_boost = CatBoostRegressor(**cat_boost_kwards)
gbr = GradientBoostingRegressor(**gbr_kwards)
xgb = XGBRegressor(**xgb_kwargs)

In [48]:
cat_boost.fit(X_train, y_train)
gbr.fit(X_train_ohe, y_train)
xgb.fit(X_train_ohe, y_train)

0:	learn: 73875.1516169	total: 66ms	remaining: 12m 5s
1:	learn: 69106.2188585	total: 129ms	remaining: 11m 51s
2:	learn: 64751.1742487	total: 198ms	remaining: 12m 5s
3:	learn: 60642.0075990	total: 283ms	remaining: 12m 57s
4:	learn: 57217.5572104	total: 349ms	remaining: 12m 48s
5:	learn: 54008.4541614	total: 412ms	remaining: 12m 34s
6:	learn: 50991.5302032	total: 497ms	remaining: 13m
7:	learn: 48123.6351630	total: 560ms	remaining: 12m 49s
8:	learn: 45826.2728633	total: 628ms	remaining: 12m 47s
9:	learn: 43500.1193021	total: 712ms	remaining: 13m 2s
10:	learn: 41441.8594670	total: 788ms	remaining: 13m 6s
11:	learn: 39457.9210652	total: 873ms	remaining: 13m 19s
12:	learn: 37779.9877165	total: 960ms	remaining: 13m 31s
13:	learn: 36231.2177764	total: 1.03s	remaining: 13m 27s
14:	learn: 34706.1576590	total: 1.09s	remaining: 13m 20s
15:	learn: 33466.4268848	total: 1.18s	remaining: 13m 27s
16:	learn: 32419.9245749	total: 1.24s	remaining: 13m 23s
17:	learn: 31167.5530048	total: 1.31s	remaining: 1

144:	learn: 10672.6579581	total: 11s	remaining: 13m 40s
145:	learn: 10657.7522403	total: 11s	remaining: 13m 40s
146:	learn: 10589.6123493	total: 11.1s	remaining: 13m 39s
147:	learn: 10516.9294757	total: 11.2s	remaining: 13m 40s
148:	learn: 10467.4486883	total: 11.3s	remaining: 13m 39s
149:	learn: 10432.7056464	total: 11.3s	remaining: 13m 38s
150:	learn: 10405.0401175	total: 11.4s	remaining: 13m 39s
151:	learn: 10340.0521590	total: 11.5s	remaining: 13m 39s
152:	learn: 10287.8012824	total: 11.6s	remaining: 13m 39s
153:	learn: 10199.3177031	total: 11.7s	remaining: 13m 40s
154:	learn: 10124.7164605	total: 11.7s	remaining: 13m 40s
155:	learn: 10060.9847772	total: 11.8s	remaining: 13m 40s
156:	learn: 10057.0362552	total: 11.9s	remaining: 13m 41s
157:	learn: 10049.5757713	total: 12s	remaining: 13m 41s
158:	learn: 10006.7178881	total: 12s	remaining: 13m 41s
159:	learn: 10004.1717696	total: 12.1s	remaining: 13m 42s
160:	learn: 9942.9268416	total: 12.2s	remaining: 13m 41s
161:	learn: 9905.254835

291:	learn: 5706.2557289	total: 22.5s	remaining: 13m 45s
292:	learn: 5696.9879178	total: 22.6s	remaining: 13m 45s
293:	learn: 5664.6739485	total: 22.7s	remaining: 13m 45s
294:	learn: 5658.5652245	total: 22.8s	remaining: 13m 46s
295:	learn: 5639.3690276	total: 22.9s	remaining: 13m 46s
296:	learn: 5637.7741184	total: 22.9s	remaining: 13m 46s
297:	learn: 5604.9798034	total: 23s	remaining: 13m 47s
298:	learn: 5572.3321081	total: 23.1s	remaining: 13m 46s
299:	learn: 5544.0424432	total: 23.2s	remaining: 13m 46s
300:	learn: 5506.5556169	total: 23.3s	remaining: 13m 47s
301:	learn: 5496.0150340	total: 23.4s	remaining: 13m 47s
302:	learn: 5460.0618304	total: 23.4s	remaining: 13m 47s
303:	learn: 5433.1766875	total: 23.5s	remaining: 13m 47s
304:	learn: 5427.0083558	total: 23.6s	remaining: 13m 47s
305:	learn: 5398.1930026	total: 23.7s	remaining: 13m 47s
306:	learn: 5372.9594643	total: 23.8s	remaining: 13m 47s
307:	learn: 5364.2222483	total: 23.8s	remaining: 13m 47s
308:	learn: 5349.8129689	total: 2

438:	learn: 3419.0044744	total: 34.4s	remaining: 13m 48s
439:	learn: 3410.7843027	total: 34.5s	remaining: 13m 48s
440:	learn: 3406.6145799	total: 34.6s	remaining: 13m 48s
441:	learn: 3392.7337350	total: 34.7s	remaining: 13m 48s
442:	learn: 3388.0848877	total: 34.8s	remaining: 13m 48s
443:	learn: 3378.2164506	total: 34.8s	remaining: 13m 48s
444:	learn: 3354.5245152	total: 34.9s	remaining: 13m 48s
445:	learn: 3336.3429631	total: 35s	remaining: 13m 48s
446:	learn: 3322.1509622	total: 35.1s	remaining: 13m 48s
447:	learn: 3295.5289647	total: 35.2s	remaining: 13m 48s
448:	learn: 3288.1387045	total: 35.2s	remaining: 13m 48s
449:	learn: 3276.4717715	total: 35.3s	remaining: 13m 47s
450:	learn: 3274.0024155	total: 35.4s	remaining: 13m 48s
451:	learn: 3265.8799444	total: 35.5s	remaining: 13m 47s
452:	learn: 3255.1636035	total: 35.6s	remaining: 13m 47s
453:	learn: 3240.2844870	total: 35.6s	remaining: 13m 48s
454:	learn: 3239.1428633	total: 35.7s	remaining: 13m 48s
455:	learn: 3220.3511544	total: 3

585:	learn: 2110.3184604	total: 46.3s	remaining: 13m 42s
586:	learn: 2100.2610056	total: 46.3s	remaining: 13m 42s
587:	learn: 2097.3091877	total: 46.4s	remaining: 13m 42s
588:	learn: 2091.4959938	total: 46.5s	remaining: 13m 42s
589:	learn: 2083.6669477	total: 46.6s	remaining: 13m 42s
590:	learn: 2077.1171020	total: 46.7s	remaining: 13m 41s
591:	learn: 2070.0025159	total: 46.8s	remaining: 13m 42s
592:	learn: 2065.1404179	total: 46.8s	remaining: 13m 41s
593:	learn: 2061.5190024	total: 46.9s	remaining: 13m 41s
594:	learn: 2054.8143798	total: 47s	remaining: 13m 41s
595:	learn: 2045.0294062	total: 47.1s	remaining: 13m 41s
596:	learn: 2038.7678907	total: 47.1s	remaining: 13m 41s
597:	learn: 2029.2413383	total: 47.2s	remaining: 13m 41s
598:	learn: 2024.4593405	total: 47.3s	remaining: 13m 40s
599:	learn: 2013.2001757	total: 47.3s	remaining: 13m 40s
600:	learn: 2009.4305306	total: 47.4s	remaining: 13m 40s
601:	learn: 2001.6455251	total: 47.5s	remaining: 13m 40s
602:	learn: 1994.8753117	total: 4

...

10520:	learn: 0.1285468	total: 13m 37s	remaining: 37.2s
10521:	learn: 0.1285454	total: 13m 37s	remaining: 37.1s
10522:	learn: 0.1268390	total: 13m 37s	remaining: 37.1s
10523:	learn: 0.1268382	total: 13m 37s	remaining: 37s
10524:	learn: 0.1268343	total: 13m 37s	remaining: 36.9s
10525:	learn: 0.1268341	total: 13m 38s	remaining: 36.8s
10526:	learn: 0.1268313	total: 13m 38s	remaining: 36.8s
10527:	learn: 0.1268137	total: 13m 38s	remaining: 36.7s
10528:	learn: 0.1268102	total: 13m 38s	remaining: 36.6s
10529:	learn: 0.1268042	total: 13m 38s	remaining: 36.5s
10530:	learn: 0.1268020	total: 13m 38s	remaining: 36.4s
10531:	learn: 0.1268010	total: 13m 38s	remaining: 36.4s
10532:	learn: 0.1267945	total: 13m 38s	remaining: 36.3s
10533:	learn: 0.1267879	total: 13m 38s	remaining: 36.2s
10534:	learn: 0.1267845	total: 13m 38s	remaining: 36.1s
10535:	learn: 0.1267816	total: 13m 38s	remaining: 36.1s
10536:	learn: 0.1267713	total: 13m 38s	remaining: 36s
10537:	learn: 0.1267699	total: 13m 38s	remaining: 35

10669:	learn: 0.1101141	total: 13m 49s	remaining: 25.7s
10670:	learn: 0.1101032	total: 13m 49s	remaining: 25.6s
10671:	learn: 0.1100975	total: 13m 49s	remaining: 25.5s
10672:	learn: 0.1100897	total: 13m 49s	remaining: 25.4s
10673:	learn: 0.1100879	total: 13m 49s	remaining: 25.3s
10674:	learn: 0.1100873	total: 13m 49s	remaining: 25.3s
10675:	learn: 0.1100866	total: 13m 49s	remaining: 25.2s
10676:	learn: 0.1100714	total: 13m 49s	remaining: 25.1s
10677:	learn: 0.1100708	total: 13m 50s	remaining: 25s
10678:	learn: 0.1100682	total: 13m 50s	remaining: 25s
10679:	learn: 0.1100636	total: 13m 50s	remaining: 24.9s
10680:	learn: 0.1100589	total: 13m 50s	remaining: 24.8s
10681:	learn: 0.1100483	total: 13m 50s	remaining: 24.7s
10682:	learn: 0.1100446	total: 13m 50s	remaining: 24.6s
10683:	learn: 0.1100170	total: 13m 50s	remaining: 24.6s
10684:	learn: 0.1087136	total: 13m 50s	remaining: 24.5s
10685:	learn: 0.1087131	total: 13m 50s	remaining: 24.4s
10686:	learn: 0.1087110	total: 13m 50s	remaining: 24

10818:	learn: 0.0940451	total: 14m 1s	remaining: 14.1s
10819:	learn: 0.0940310	total: 14m 1s	remaining: 14s
10820:	learn: 0.0940301	total: 14m 1s	remaining: 13.9s
10821:	learn: 0.0940295	total: 14m 1s	remaining: 13.8s
10822:	learn: 0.0940274	total: 14m 1s	remaining: 13.8s
10823:	learn: 0.0940251	total: 14m 1s	remaining: 13.7s
10824:	learn: 0.0940216	total: 14m 1s	remaining: 13.6s
10825:	learn: 0.0940207	total: 14m 1s	remaining: 13.5s
10826:	learn: 0.0927288	total: 14m 1s	remaining: 13.4s
10827:	learn: 0.0927212	total: 14m 1s	remaining: 13.4s
10828:	learn: 0.0927190	total: 14m 1s	remaining: 13.3s
10829:	learn: 0.0927189	total: 14m 1s	remaining: 13.2s
10830:	learn: 0.0927168	total: 14m 2s	remaining: 13.1s
10831:	learn: 0.0927165	total: 14m 2s	remaining: 13.1s
10832:	learn: 0.0927087	total: 14m 2s	remaining: 13s
10833:	learn: 0.0927022	total: 14m 2s	remaining: 12.9s
10834:	learn: 0.0927016	total: 14m 2s	remaining: 12.8s
10835:	learn: 0.0927005	total: 14m 2s	remaining: 12.8s
10836:	learn: 

10968:	learn: 0.0878353	total: 14m 13s	remaining: 2.41s
10969:	learn: 0.0878302	total: 14m 13s	remaining: 2.33s
10970:	learn: 0.0878298	total: 14m 13s	remaining: 2.25s
10971:	learn: 0.0868712	total: 14m 13s	remaining: 2.18s
10972:	learn: 0.0868699	total: 14m 13s	remaining: 2.1s
10973:	learn: 0.0868693	total: 14m 13s	remaining: 2.02s
10974:	learn: 0.0868654	total: 14m 13s	remaining: 1.94s
10975:	learn: 0.0868637	total: 14m 13s	remaining: 1.87s
10976:	learn: 0.0864771	total: 14m 13s	remaining: 1.79s
10977:	learn: 0.0864740	total: 14m 14s	remaining: 1.71s
10978:	learn: 0.0864733	total: 14m 14s	remaining: 1.63s
10979:	learn: 0.0864721	total: 14m 14s	remaining: 1.55s
10980:	learn: 0.0864679	total: 14m 14s	remaining: 1.48s
10981:	learn: 0.0864643	total: 14m 14s	remaining: 1.4s
10982:	learn: 0.0864623	total: 14m 14s	remaining: 1.32s
10983:	learn: 0.0864595	total: 14m 14s	remaining: 1.24s
10984:	learn: 0.0864579	total: 14m 14s	remaining: 1.17s
10985:	learn: 0.0864555	total: 14m 14s	remaining: 

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=<function mean_squared_error at 0x7f4ab8b03520>,
             gamma=0, gpu_id=-1, grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_to_onehot=4, max_delta_step=0, max_depth=3, max_leaves=0,
             min_child_weight=1e-09, missing=nan, monotone_constraints='()',
             n_estimators=11000, n_jobs=0, num_parallel_tree=1,
             predictor='auto', random_state=10, reg_alpha=0, reg_lambda=0.1, ...)

In [54]:
pred = np.zeros((3, X_test.shape[0]))

pred[0] = cat_boost.predict(X_test)
pred[1] = gbr.predict(X_test_ohe)
pred[2] = xgb.predict(X_test_ohe)

In [55]:
sub_name = "data/submission_18.csv"
pd.DataFrame(pred.mean(axis=0), 
            index=range(1461, len(df)+1), 
            columns=['SalePrice']).reset_index().\
            rename(columns={'index': 'id'}).to_csv(sub_name, index=False)

# Score: 